<a href="https://colab.research.google.com/github/AyushSaini0008/Skygeni_Assignment/blob/main/skygeni_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Skygeni Shortlisting & Assignment for Data Engineering Role**

---



# **Q1 : How many finance lending and blockchain clients does the organization have?**

In [ ]:
import plotly.express as px
import pandas as pd

#load client data
industry_clients = pd.read_csv('industry_client_details.csv')

#count the number of clients in each industry
industry_counts = industry_clients['industry'].value_counts().reset_index()

#rename the columns
industry_counts.columns = ['industry', 'count']

#create a pie chart for visulization
fig = px.pie(
    industry_counts,                #data for the pie chart
    values='count',                  #values that represent the size of each slice
    names='industry',                #labels for each slice
    title='Proportion of Clients by Industry',#title of the pie chart
    color='industry',                #color each slice by the industry
)

#display the value on each pie slice
fig.update_traces(
    textinfo='value'  #shows the count of clients on each slice
)

#final layout
fig.update_layout(
    showlegend=True,                #show the legend
    legend_title_text='Industry',   #title for the legend
    title_x=0.5                     #center the title horizontally
)

#display the pie chart
fig.show()


# **Q2 : Which industry in the organization has the highest renewal rate?**

In [ ]:
import pandas as pd
import plotly.express as px

#reading the data
subscription = pd.read_csv('subscription_information.csv')
industry_clients = pd.read_csv('industry_client_details.csv')

# convert renewed column to binary
# this handles both numeric and string formats
if pd.api.types.is_numeric_dtype(subscription['renewed']):
    subscription['renewed'] = subscription['renewed'].astype(int)
else:
    subscription['renewed'] = subscription['renewed'].str.lower().map({'yes': 1, 'no': 0})

#merging the subscription data with industry client details using client_id as the key
merged = pd.merge(subscription, industry_clients, on='client_id', how='inner')

#group the merged data by industry and calculate the renewal rate
#the renewal rate is the mean of the renewed column
renewal_rates = merged.groupby('industry')['renewed'].mean().reset_index()
renewal_rates['renewed'] = renewal_rates['renewed'] * 100

#sort the industries by renewal rate from highest to lowest
renewal_rates = renewal_rates.sort_values(by='renewed', ascending=False)

#highlight the industry with the highest renewal rate
highlight_industry = renewal_rates.iloc[0]['industry']  #industry with the highest renewal rate

#create a bar graph
fig = px.bar(
    renewal_rates,
    x='industry',                    #X-axis: Industries
    y='renewed',                     #Y-axis: Renewal rate (%)
    color='industry',                #color the bars by industry
    title="Renewal Rate by Industry", #chart title
    text=renewal_rates['renewed'].apply(lambda x: f"{x:.1f}%"),  #display renewal rate in percentage format
    labels={'renewed': 'Renewal Rate (%)', 'industry': 'Industry'}
)

#customizing the layout
fig.update_traces(textposition='outside')  #place the text outside the bars
fig.update_layout(
    uniformtext_minsize=8,                #set minimum font size for the text
    uniformtext_mode='hide',              #hide the text if it doesn't fit inside the bars
    xaxis_tickangle=45,                   #rotate x-axis labels
    title_x=0.5                           #center the title
)

#display the chart
fig.show()


#  Q3.What was the average inflation rate when their subscriptions were renewed?**bold text**

In [ ]:
import pandas as pd
import plotly.express as px

#reading the data
financial = pd.read_csv('finanical_information.csv')
subscription = pd.read_csv('subscription_information.csv')

#convert date columns to datetime format
financial['start_date'] = pd.to_datetime(financial['start_date'])
financial['end_date'] = pd.to_datetime(financial['end_date'])
subscription['end_date'] = pd.to_datetime(subscription['end_date'])

#ensure renewed column is in binary format
if not pd.api.types.is_numeric_dtype(subscription['renewed']):
    subscription['renewed'] = subscription['renewed'].str.lower().map({'yes': 1, 'no': 0})

#filter only the subscriptions that have been renewed
renewed_subscriptions = subscription[subscription['renewed'] == 1].copy()

#merge the renewed subscriptions with the financial data based on date
#we use merge_asof to match each renewal with the correct financial period
merged_renewed = pd.merge_asof(
    renewed_subscriptions.sort_values('end_date'),
    financial.sort_values('start_date'),
    left_on='end_date',
    right_on='start_date',
    direction='backward',  #align with the last financial period before the renewal date
    suffixes=('_sub', '_fin')
)

#Filter out rows where the subscription end date is beyond the financial period
valid = merged_renewed[merged_renewed['end_date_sub'] <= merged_renewed['end_date_fin']]

#Drop rows with missing inflation rate
valid = valid.dropna(subset=['inflation_rate'])

#calculate and print the average inflation rate during the renewal periods
average_inflation = valid['inflation_rate'].mean()
print(f"Q3: Average inflation rate during renewals: {average_inflation:.2f}%")

#create a line graph
fig_line = px.line(
    valid,
    x='end_date_sub',  #use the subscription end date for the x-axis
    y='inflation_rate',  #use the inflation rate for the y-axis
    markers=True,  #add markers to the plot
    title="Inflation Rate During Renewals Over Time",
    labels={
        'end_date_sub': 'Renewal Date',
        'inflation_rate': 'Inflation Rate (%)'
    },
    template='plotly_white'
)

#customize the layout
fig_line.update_layout(
    hovermode='x unified',  #show hover information for the entire x-axis value
    title_font_size=20,  #increase font size for title
    xaxis_title_font_size=16,  #adjust font size for x-axis title
    yaxis_title_font_size=16  #adjust font size for y-axis title
)

#display the plot
fig_line.show()


Q3: Average inflation rate during renewals: 4.31%


# **Q4.What is the median amount paid each year for all payment methods?**

In [ ]:
import pandas as pd
import plotly.express as px

#read payment data
payment = pd.read_csv('payment_information.csv')

#parse 'payment_date' column into datetime and extract the year
payment['payment_date'] = pd.to_datetime(payment['payment_date'], format='%m/%d/%Y')
payment['year'] = payment['payment_date'].dt.year  # Extract the year from the payment date

#group the data by year and payment_method and calculate the median amount_paid
# This gives us the median payment for each payment method by year
median_payments = payment.groupby(['year', 'payment_method'])['amount_paid'].median().reset_index()

#display the result as a markdown table
print("Q4: Median amount paid by year and payment method:")
print(median_payments.to_markdown(index=False))

#create a line graph
fig = px.line(
    median_payments,  #the data to be visualized
    x='year',         #X-axis: Year of payment
    y='amount_paid',  #Y-axis: Median amount paid
    color='payment_method',  #different lines for each payment method
    markers=True,           #add markers to each data point f
    title="Median Amount Paid Per Year by Payment Method",
    labels={
        'year': 'Year',
        'amount_paid': 'Median Amount Paid',
        'payment_method': 'Payment Method'
    }
)

#display the plot
fig.show()


Q4: Median amount paid by year and payment method:
|   year | payment_method   |   amount_paid |
|-------:|:-----------------|--------------:|
|   2018 | Bank Transfer    |        281.65 |
|   2018 | Check            |        216.6  |
|   2018 | Credit Card      |        229.15 |
|   2019 | Bank Transfer    |        184.2  |
|   2019 | Check            |        410.2  |
|   2019 | Credit Card      |        401.9  |
|   2020 | Bank Transfer    |        225.1  |
|   2020 | Check            |        413.1  |
|   2020 | Credit Card      |        285.25 |
|   2021 | Bank Transfer    |        255.3  |
|   2021 | Check            |        435.1  |
|   2021 | Credit Card      |        208.7  |
|   2022 | Bank Transfer    |        196.5  |
|   2022 | Check            |        275.5  |
|   2022 | Credit Card      |        326.2  |
